# 03. LLM Enhanced Training

이 노트북은 LLM 생성 데이터로 enhanced 모델을 학습하고 성능을 비교합니다.

## 전제조건
먼저 다음 노트북들을 순서대로 실행해야 합니다:
1. `01_neural_sparse_base_training.ipynb` - 기본 모델 학습
2. `02_llm_synthetic_data_generation.ipynb` - LLM 데이터 생성

## 목표
- 기본 데이터 + LLM 생성 데이터 결합
- Enhanced Neural Sparse 모델 학습
- Base vs Enhanced 모델 성능 비교

## 출력 데이터
모든 데이터는 `dataset/enhanced_model/` 디렉토리에 저장됩니다:
- `neural_sparse_v2_model/`: Enhanced 모델
- `evaluation/performance_comparison.json`: 성능 비교 결과

In [ ]:
# DatasetManager 초기화
from src.dataset_manager import DatasetManager

dm = DatasetManager(base_path="dataset")
print("✅ DatasetManager initialized")
print(f"📁 Base path: {dm.base_path.absolute()}")

In [ ]:
# 필수 데이터 파일 확인
required_files = [
    ("base_model", "korean_documents.json"),
    ("base_model", "qd_pairs_base.pkl"),
    ("base_model", "neural_sparse_v1_model"),
    ("llm_generated", "synthetic_qd_pairs.pkl"),
    ("llm_generated", "enhanced_synonyms.json"),
]

if not dm.check_dependencies(required_files):
    raise RuntimeError(
        "Required data files not found. "
        "Please run notebooks 1 and 2 first."
    )

print("\n✅ Ready to proceed with enhanced training")

In [ ]:
# 이전 노트북들에서 생성된 데이터 로드
print("Loading data from previous notebooks...\n")

# From notebook 1
korean_documents = dm.load_json("korean_documents.json", "base_model")
qd_pairs_base = dm.load_pickle("qd_pairs_base.pkl", "base_model")

# From notebook 2
synthetic_qd_pairs = dm.load_pickle("synthetic_qd_pairs.pkl", "llm_generated")
enhanced_synonyms = dm.load_json("enhanced_synonyms.json", "llm_generated")

print(f"\n✅ Loaded {len(korean_documents)} documents")
print(f"✅ Loaded {len(qd_pairs_base)} base QD pairs")
print(f"✅ Loaded {len(synthetic_qd_pairs)} synthetic QD pairs")
print(f"✅ Loaded enhanced synonyms with {len(enhanced_synonyms)} entries")

In [ ]:
# Base 모델 로드 (성능 비교용)
from src.opensearch_sparse_encoder import OpenSearchSparseEncoder

base_model, base_tokenizer = dm.load_model(
    OpenSearchSparseEncoder,
    "neural_sparse_v1_model",
    "base_model"
)

print("✅ Base model loaded for comparison")

## 🆕 16. 합성 데이터 포함 모델 재학습

기존 데이터와 LLM으로 생성한 합성 데이터를 결합하여 모델을 재학습합니다.

In [ ]:
print("\n" + "="*70)
print("🔄 섹션 16: 합성 데이터 포함 모델 재학습")
print("="*70)

# 기존 데이터 + 합성 데이터 병합
combined_qd_pairs = korean_data['qd_pairs'][:10000] + synthetic_pairs

print(f"\n📊 학습 데이터 통계:")
print(f"   기존 데이터: {len(korean_data['qd_pairs'][:10000]):,}개")
print(f"   합성 데이터: {len(synthetic_pairs):,}개")
print(f"   총 데이터: {len(combined_qd_pairs):,}개")
print(f"   증가율: {len(synthetic_pairs) / len(korean_data['qd_pairs'][:10000]) * 100:.1f}%")

# Negative sampling 적용
print("\n🔄 Negative sampling 적용 중...")
augmented_pairs_v2 = add_negative_samples(
    combined_qd_pairs,
    korean_data['documents'],
    num_negatives=2
)

# Train/Val split
train_pairs_v2, val_pairs_v2 = train_test_split(augmented_pairs_v2, test_size=0.1, random_state=42)

print(f"\n📊 재학습 데이터셋 분할:")
print(f"   Train: {len(train_pairs_v2):,} pairs")
print(f"   Val: {len(val_pairs_v2):,} pairs")

# 데이터셋 및 로더 생성
train_dataset_v2 = SparseEncodingDataset(train_pairs_v2, tokenizer, MAX_LENGTH)
val_dataset_v2 = SparseEncodingDataset(val_pairs_v2, tokenizer, MAX_LENGTH)

train_loader_v2 = DataLoader(train_dataset_v2, batch_size=BATCH_SIZE, shuffle=True)
val_loader_v2 = DataLoader(val_dataset_v2, batch_size=BATCH_SIZE, shuffle=False)

print(f"\n✓ 재학습 데이터 로더 생성 완료")
print(f"   Train batches: {len(train_loader_v2):,}")
print(f"   Val batches: {len(val_loader_v2):,}")

# 모델 재초기화 (새로 학습)
print("\n🔄 모델 재초기화 중...")
doc_encoder_v2 = OpenSearchDocEncoder(MODEL_NAME)
doc_encoder_v2 = doc_encoder_v2.to(device)

if len(tokenizer) > doc_encoder_v2.vocab_size:
    doc_encoder_v2.bert.resize_token_embeddings(len(tokenizer))
    doc_encoder_v2.vocab_size = len(tokenizer)

print(f"✓ 모델 v2 초기화 완료")

# Optimizer & Scheduler for v2
optimizer_v2 = AdamW(doc_encoder_v2.parameters(), lr=LEARNING_RATE)
total_steps_v2 = len(train_loader_v2) * NUM_EPOCHS
scheduler_v2 = get_linear_schedule_with_warmup(
    optimizer_v2,
    num_warmup_steps=WARMUP_STEPS,
    num_training_steps=total_steps_v2
)

# 학습 히스토리 v2
history_v2 = {
    'train_loss': [],
    'val_loss': [],
    'ranking_loss': [],
    'l0_loss': [],
    'idf_penalty': []
}

print("\n" + "="*70)
print("🚀 LLM 확장 모델 학습 시작!")
print("="*70)

# 학습 실행 (v2)
best_val_loss_v2 = float('inf')
best_model_path_v2 = "./models/best_korean_neural_sparse_encoder_v2_llm.pt"

for epoch in range(NUM_EPOCHS):
    print(f"\n{'='*70}")
    print(f"[V2 LLM] Epoch {epoch + 1}/{NUM_EPOCHS}")
    print(f"{'='*70}")
    
    # 학습
    train_loss, ranking_loss, l0_loss, idf_penalty = train_epoch(
        doc_encoder_v2, train_loader_v2, optimizer_v2, scheduler_v2,
        idf_id_dict, tokenizer, device
    )
    
    # 검증
    val_loss = evaluate(doc_encoder_v2, val_loader_v2, idf_id_dict, tokenizer, device)
    
    # 기록
    history_v2['train_loss'].append(train_loss)
    history_v2['val_loss'].append(val_loss)
    history_v2['ranking_loss'].append(ranking_loss)
    history_v2['l0_loss'].append(l0_loss)
    history_v2['idf_penalty'].append(idf_penalty)
    
    print(f"\n📊 [V2] Epoch {epoch + 1} 결과:")
    print(f"  Train Loss: {train_loss:.4f}")
    print(f"  Val Loss: {val_loss:.4f}")
    print(f"  Ranking Loss: {ranking_loss:.4f}")
    print(f"  L0 Loss: {l0_loss:.2f}")
    print(f"  IDF Penalty: {idf_penalty:.4f}")
    
    # 베스트 모델 저장
    if val_loss < best_val_loss_v2:
        best_val_loss_v2 = val_loss
        torch.save({
            'epoch': epoch,
            'model_state_dict': doc_encoder_v2.state_dict(),
            'optimizer_state_dict': optimizer_v2.state_dict(),
            'val_loss': val_loss,
            'config': {
                'model_name': MODEL_NAME,
                'vocab_size': len(tokenizer),
                'max_length': MAX_LENGTH,
            }
        }, best_model_path_v2)
        print(f"  ✓ 베스트 모델 v2 저장! (val_loss: {val_loss:.4f})")

print("\n" + "="*70)
print("✅ LLM 확장 모델 학습 완료!")
print("="*70)
print(f"Best Validation Loss (V2): {best_val_loss_v2:.4f}")

## 🆕 17. 성능 비교 분석 (기존 vs LLM 확장)

기존 모델과 LLM으로 확장한 모델의 성능을 비교합니다.

In [ ]:
print("\n" + "="*70)
print("📊 섹션 17: 성능 비교 분석 (기존 vs LLM 확장)")
print("="*70)

# 기존 모델 vs LLM 확장 모델 비교
comparison_results = {
    '항목': [
        '모델 버전',
        '학습 데이터 (pairs)',
        '동의어 사전 (항목)',
        'Training Loss',
        'Validation Loss',
        '성능 개선율'
    ],
    '기존 모델 (V1)': [
        'v1 (baseline)',
        f"{len(korean_data['qd_pairs'][:10000]):,}",
        f"{len(get_default_korean_english_pairs())}",
        f"{history['train_loss'][-1]:.4f}",
        f"{best_val_loss:.4f}",
        "baseline"
    ],
    'LLM 확장 모델 (V2)': [
        'v2_llm',
        f"{len(combined_qd_pairs):,}",
        f"{len(enhanced_bilingual_dict):,}",
        f"{history_v2['train_loss'][-1]:.4f}",
        f"{best_val_loss_v2:.4f}",
        f"{((best_val_loss - best_val_loss_v2) / best_val_loss * 100):.2f}%"
    ]
}

df_comparison = pd.DataFrame(comparison_results)

print("\n📊 모델 성능 비교표:")
print("="*70)
print(df_comparison.to_string(index=False))
print("="*70)

# 학습 곡선 비교
print("\n📈 학습 곡선 비교:")
print("\n[V1 - 기존 모델]")
for epoch, (train_loss, val_loss) in enumerate(zip(history['train_loss'], history['val_loss']), 1):
    print(f"  Epoch {epoch}: Train={train_loss:.4f}, Val={val_loss:.4f}")

print("\n[V2 - LLM 확장 모델]")
for epoch, (train_loss, val_loss) in enumerate(zip(history_v2['train_loss'], history_v2['val_loss']), 1):
    print(f"  Epoch {epoch}: Train={train_loss:.4f}, Val={val_loss:.4f}")

# 개선 사항 요약
print("\n" + "="*70)
print("✅ 주요 개선 사항")
print("="*70)

data_increase = len(synthetic_pairs) / len(korean_data['qd_pairs'][:10000]) * 100
synonym_increase = (len(enhanced_bilingual_dict) - len(get_default_korean_english_pairs())) / len(get_default_korean_english_pairs()) * 100
loss_improvement = (best_val_loss - best_val_loss_v2) / best_val_loss * 100

print(f"\n1️⃣ 학습 데이터 확장")
print(f"   - 기존: {len(korean_data['qd_pairs'][:10000]):,}개")
print(f"   - LLM 합성: +{len(synthetic_pairs):,}개")
print(f"   - 증가율: +{data_increase:.1f}%")

print(f"\n2️⃣ 동의어 사전 품질 향상")
print(f"   - 기존: {len(get_default_korean_english_pairs())}개 (수동)")
print(f"   - LLM 검증: {len(enhanced_bilingual_dict):,}개")
print(f"   - 증가율: +{synonym_increase:.1f}%")

print(f"\n3️⃣ 검증 손실 개선")
print(f"   - 기존 모델: {best_val_loss:.4f}")
print(f"   - LLM 확장: {best_val_loss_v2:.4f}")
print(f"   - 개선율: {loss_improvement:.2f}%")

if best_val_loss_v2 < best_val_loss:
    print(f"\n✅ LLM 확장 모델이 기존 모델보다 우수합니다!")
else:
    print(f"\n⚠️  기존 모델과 비슷하거나 더 많은 학습이 필요할 수 있습니다.")
    print(f"   - 더 많은 합성 데이터 생성")
    print(f"   - 에포크 수 증가")
    print(f"   - 하이퍼파라미터 튜닝 권장")

print("\n" + "="*70)
print("🎉 모든 실험 완료!")
print("="*70)

---

# 데이터 저장 (최종 결과)

Enhanced 모델과 평가 결과를 `dataset/enhanced_model/` 디렉토리에 저장합니다.

In [ ]:
# 1. Enhanced Neural Sparse 모델 저장
dm.save_model(
    enhanced_model,
    tokenizer,
    "neural_sparse_v2_model",
    "enhanced_model"
)

In [ ]:
# 2. 성능 비교 결과 저장
dm.save_json(
    performance_comparison,
    "performance_comparison.json",
    "enhanced_model/evaluation"
)

In [ ]:
# 전체 데이터셋 요약
dm.print_summary()

---

## ✅ Notebook 3 완료 - 전체 파이프라인 완성!

Enhanced 모델과 평가 결과가 `dataset/enhanced_model/`에 저장되었습니다.

### 완료된 작업
1. ✅ Base 모델 학습 (Notebook 1)
2. ✅ LLM 합성 데이터 생성 (Notebook 2)
3. ✅ Enhanced 모델 학습 및 비교 (Notebook 3)

### 결과 확인
- Base Model: `dataset/base_model/neural_sparse_v1_model/`
- Enhanced Model: `dataset/enhanced_model/neural_sparse_v2_model/`
- Performance: `dataset/enhanced_model/evaluation/performance_comparison.json`

### 다음 단계
이제 OpenSearch에 모델을 배포하고 실제 검색 서비스에서 사용할 수 있습니다!